In [59]:
import numpy as np
import pandas as pd

**EDA**

In [60]:
df = pd.read_csv('../input/fake-news-prediction/train.csv')

In [61]:
df.head()

In [62]:
df.shape

In [63]:
df.describe()

In [64]:
df.info()

In [65]:
df.isnull().sum()

In [66]:
# Drop Nan Values
df = df.dropna()

In [67]:
df.head()

In [68]:
# Get The Independent  Features
X = df.drop('label',axis=1)

In [69]:
print(X)

In [70]:
X.shape

In [71]:
# Get The Dependent Feature
y = df['label']

In [72]:
print(y)

In [73]:
y.shape

In [74]:
import tensorflow as tf

In [75]:
tf.__version__

In [76]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense

**One Hot Representation**

In [77]:
# Vocablary Size
voc_size = 5000

In [78]:
dataset = X.copy()

In [79]:
dataset['title'][1]

In [80]:
dataset

In [81]:
dataset.reset_index(inplace = True)

In [82]:
dataset

In [83]:
import nltk
import re
from nltk.corpus import stopwords

In [84]:
nltk.download('stopwords')

In [85]:
# Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(dataset)):
    review = re.sub('[^a-zA-Z]', ' ', dataset['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [86]:
corpus

In [87]:
onehot_rep =[one_hot(words,voc_size) for words in corpus]

In [88]:
onehot_rep

**Embedding Representation**

In [89]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep,padding = 'post',maxlen = sent_length)
print(embedded_docs)

In [90]:
embedded_docs[0]

**Creating LSTM Model**

In [91]:
embedding_features = 40 #For Features In Every Word In Embedded_docs
model = Sequential()
model.add(Embedding(voc_size,embedding_features,input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer = 'adam',loss= 'binary_crossentropy',metrics=['accuracy'])

In [92]:
model.summary()

In [93]:
len(embedded_docs),y.shape

In [94]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [95]:
X_final.shape , y_final.shape

In [96]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size = 0.33,random_state = 42)

**Model Training**

In [97]:
model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 20 ,batch_size= 64)

**Performance  Metrics And Accuracy**

In [98]:
y_pred = model.predict(X_test)

In [99]:
y_pred = np.where(y_pred>0.6,1,0) #AUC ROC Curve

In [100]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

In [101]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [102]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))